In [2]:
import datetime
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.max_columns',7)
pd.set_option('display.max_rows',10)
pd.set_option('display.width',250)
%matplotlib inline
from re import match
import os
import json
import pymysql

In [ ]:
import pymysql
config = {     # DB연동을 위한 환경변수
    'host':'127.0.0.1', # 각자의 접속DB주소, 127.0.0.1
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

class Product:
    def __init__(self,config):
        self.config = config
        self.conn = pymysql.connect(**self.config)
        self.cursor = self.conn.cursor()
        #print('객체 생성 확인 : ',self.conn)
        #print('커서 객체 확인 : ',self.cursor)
        
    def createTable(self):
        sql = 'CREATE TABLE if not exists goods(CODE INT PRIMARY KEY,NAME VARCHAR(30) NOT NULL,su INT default 0,dan INT default 0);'
        self.cursor.execute(sql)
        self.conn.commit()
    
    def insert(self,code,name,su,dan):
        sql = f"insert into goods values({code},'{name}',{su},{dan})"
        self.cursor.execute(sql)
        self.conn.commit()    
    
    def selectAll(self):    
        sql = sql = "select * from goods"
        self.cursor.execute(sql)
        lst = self.cursor.fetchall()
        print("code\tname\t\t수량\t단가")
        for row in lst:
            print("{}\t{}\t\t{}\t{}".format(row[0],row[1],row[2],row[3]))
        print('총 상품 수량 : ',len(lst))
    
    def selectOne_n(self,name):
        sql = f"select * from goods where name like '%{name}%'"
        self.cursor.execute(sql)
        rows = self.cursor.fetchall()
        if rows:
            print("code\tname\t\t수량\t단가")
            for row in rows:
                print("{}\t{}\t\t{}\t{}".format(row[0],row[1],row[2],row[3])) 
        else:
            print('해당 상품이 없습니다.')

    def selectOne_c(self,code):
        sql = f"select * from goods where code like '%{code}%'"
        self.cursor.execute(sql)
        rows = self.cursor.fetchall()
        if rows:
            print("code\tname\t\t수량\t단가")
            for row in rows:
                print("{}\t{}\t\t{}\t{}".format(row[0],row[1],row[2],row[3])) 
        else:
            print('해당 상품이 없습니다.')
    
    def update(self,code):
        conn = pymysql.connect(**self.config)
        cursor = conn.cursor()
        name = input('변경할 상품명을 입력하세요 : ')
        su = int(input('변경할 수량을 입력하세요 : '))
        dan = int(input('변경할 단가를 입력하세요 : '))
        sql = f"update goods set name={name},su={su},dan={dan} where code={code}"
        cursor.execute(sql)
        conn.commit()
        
    def delete(self,code):
        conn = pymysql.connect(**self.config)
        cursor = conn.cursor()
        sql = f"delete from goods where code={code}"
        del_sel = input('정말 삭제하시겠습니까? (Y/n) : ')
        if del_sel.lower() == 'y':
            cursor.execute(sql)
            conn.commit()
        else:
            print('삭제를 취소합니다.')


while True:
    x = Product(config)
    os.system('cls')
    print('-----메뉴-----'+'\n1. 상품등록\n2. 상품조회(전체)\n3. 상품조회(상품명검색)\n4. 상품수정\n5. 상품삭제\n0.프로그램 종료')
    
    x.createTable()
    
    sel = int(input('메뉴를 선택하세요'))
    
    match sel:
        case 1:
            code = int(input('코드를 입력하세요 : '))
            name = input('상품명을 입력하세요 : ')
            su = int(input('수량을 입력하세요 : '))
            dan = int(input('단가를 입력하세요 : '))
            x.insert(code,name,su,dan)
        case 2:
            print('전체상품조회')
            x.selectAll()
        case 3:
            print('개별상품조회')
            name = input('\n찾을 상품명을 입력하세요 : ')
            x.selectOne(name)
        case 4:
            print('상품 정보 수정')
            code = int(input('수정할 상품의 코드를 입력하세요 : '))
            x.update(code)
        case 5:
            print('상품 정보 삭제')
            code = int(input('삭제할 상품의 코드를 입력하세요 : '))
            x.delete(code)
        case 0:
            print('프로그램을 종료합니다.')
            break
        case _:
            print('메뉴 선택을 잘못했습니다.')
            os.system('pause')

In [ ]:
### file to DB
# 1. CSV to table
#   csv파일은 콤마를 기준으로 열단위 자료를 작성하는 파일로 데이터데이스의 테이블 구조와 호환성이 높다

In [ ]:
## 예제
# 1단계 : 테이블 생성 -> 레코드 추가
# 1. CSV파일을 읽기
bmi = pd.read_csv('data/bmi.csv')
bmi
# 2. 각 컬럼 추출
height = bmi['height']
weight = bmi['weight']
label = bmi['label']

config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    # 테이블 조회
    cursor.execute('show tables')
    tables = cursor.fetchall()
    
    # 스위칭 기법
    sw = False
    for table in tables:
        if table[0] == 'bmi_tab':
            sw = True   # table이 있는 경우 swapping
    
    # 테이블 생성
    if not sw:
        print('테이블 없음')
        sql = """
            create table bmi_tab(
                height int not null,
                weight int not null,
                label varchar(15) not null
            )
        """
        cursor.execute(sql)
    
    # 레코드 조회
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall()
    
    if rows:    # 레코드가 있는 경우 -> 레코드 조회
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}")
        print('전체 레코드 수 : ',len(rows))
    else:       # 레코드가 없는 경우 -> 레코드 추가
        print("100 레코드 추가")
        for i in range(100):
            h = height[i]
            w = weight[i]
            lab = label[i]
            cursor.execute(f"insert into bmi_tab values ({h},{w},'{lab}')")
            conn.commit()
except Exception as e:
    print('에러발생 : ',e)
finally:
    cursor.close()
    conn.close()

In [11]:
score = pd.read_csv('data/score.csv')
score

no = score['no']
kor = score['kor']
eng = score['eng']
mat = score['mat']
dept = score['dept']

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    ##
    cursor.execute("show tables")
    tables = cursor.fetchall()
    
    ## 
    sw = False
    for table in tables:
        if table[0] == 'score_tab':
            sw = True
    
    ##
    if not sw:
        print('테이블 없음')
        sql = """
            create table score_tab(
                no int not null,
                kor int not null,
                eng int not null,
                mat int not null,
                dept int not null
            )
        """
        cursor.execute(sql)
    
    ## 
    cursor.execute("select * from score_tab")
    rows = cursor.fetchall()
    
    ##
    if rows:
        print('번호\t국어\t영어\t수학\t반')
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}")
        print('전체 레코드 수 : ',len(rows))
    else:
        print('레코드 추가')
        for i in range(len(score)):
            n = no[i]
            k = kor[i]
            e = eng[i]
            m = mat[i]
            d = dept[i]
            cursor.execute(f"insert into score_tab values ({n},{k},{e},{m},{d})")
            conn.commit()
except Exception as e:
    print('에러발생 : ',e)
finally:
    cursor.close()
    conn.close()

번호	국어	영어	수학	반
101	88	85	86	101
102	70	90	92	101
103	90	45	55	101
104	55	52	74	102
105	65	85	96	102
106	75	66	41	102
107	62	75	52	103
108	40	51	85	103
109	75	90	63	101
110	50	62	75	102
111	80	55	45	103
112	95	65	85	103
113	82	45	64	101
114	88	78	82	102
115	56	75	75	103
전체 레코드 수 :  15


In [ ]:
### JSON to table
# json은 {키:값} 형식으로 제공하는 파일
# 1. json형식의 데이터를 디코딩
# 2. 데이터 프레임 형식으로 변경하면 테이블 구조의 컬럼 단위 저장

In [ ]:
# df = pd.read_json('data/labels.json')

# json 파일로드
file = open('data/labels.json','r',encoding='utf-8')

# decoding : json -> dict
lines = json.load(file)

df = pd.DataFrame(lines)
df

config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    ## 테이블 확인
    cursor.execute("show tables")
    tables = cursor.fetchall()
    
    ## 스위칭
    sw = False
    for table in tables:
        if table[0] == 'labels':
            sw = True
    
    ## 테이블 생성
    if not sw:
        print('테이블 없음')
        sql = """
        create table labels(
            id int not null,
            url varchar(150) not null,
            name varchar(50) not null,
            color varchar(50) not null,
            de char(5)
            )
        """
        cursor.execute(sql)
    
    # 레코드 조회
    cursor.execute("select * from labels")
    rows = cursor.fetchall()
    if rows:
        print("labels 조회")
        for row in rows:
            print(row[0],row[1],row[2],row[3],row[4])
        print('전체 레코드 수 : ',len(rows))
    else:
        for i in range(30):
            uid = df.id[i]
            url = df.url[i]
            name = df.name[i]
            color = df.color[i]
            de = str(df.default[i])
            sql = f"insert into labels values ({uid},'{url}','{name}','{color}','{de}')"
            cursor.execute(sql)
            conn.commit()
except Exception as e:
    print('에러발생',e)
finally:
    cursor.close()
    conn.close()

In [ ]:
### emp.csv 파일을 처리하여 Json데이터로 변환 저장하세요
# emp.csv -> dict -> json
## 저장된 json파일을 불러와서 DB작업을 하세요

In [46]:
emp = pd.read_csv('data/emp.csv')

lst = [{'No':int(emp.No[i]),'name':emp.Name[i],'Pay':int(emp.Pay[i])} for i in range(len(emp))]

print(json.dumps(lst,ensure_ascii=False,indent=4))
with open('data/emp.json','w',encoding='utf-8') as f:
    json.dump(lst,f,ensure_ascii=False,indent=4)
emp_j = pd.read_json('data/emp.json')
no = emp_j.No
config = {
    'host':'127.0.0.1',
    'user':'scott',
    'password':'tiger',
    'database':'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    ## 테이블 확인
    cursor.execute("show tables")
    tables = cursor.fetchall()
    
    ## 스위칭
    sw = False
    for table in tables:
        if table[0] == 'emp_tab':
            sw = True
    
    ## 테이블 생성
    if not sw:
        print('테이블 없음')
        sql = """
        create table emp_tab(
            no int primary key,
            name varchar(50) not null,
            pay int not null
            )
        """
        cursor.execute(sql)
    
    # 레코드 조회
    cursor.execute("select * from emp_tab")
    rows = cursor.fetchall()
    if rows:
        print("emp_tab조회")
        for row in rows:
            print(row[0],row[1],row[2])
        print('전체 레코드 수 : ',len(rows))
    else:
        for i in range(30):
            uid = df.id[i]
            url = df.url[i]
            name = df.name[i]
            
            sql = f"insert into emp_tab values ({No},'{name}','{pay}')"
            cursor.execute(sql)
            conn.commit()
except Exception as e:
    print('에러발생',e)
finally:
    cursor.close()
    conn.close()


[
    {
        "No": 101,
        "name": "홍길동",
        "Pay": 150
    },
    {
        "No": 102,
        "name": "이순신",
        "Pay": 450
    },
    {
        "No": 103,
        "name": "강감찬",
        "Pay": 500
    },
    {
        "No": 104,
        "name": "유관순",
        "Pay": 350
    },
    {
        "No": 105,
        "name": "김유신",
        "Pay": 400
    }
]
